In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as plt #Plotting lib
import seaborn as sbs # Visualization
import scipy # Stats
from sklearn.preprocessing import LabelEncoder # For encoding categorical variables
from sklearn.model_selection import train_test_split # spliting into train test
from sklearn.linear_model import LinearRegression # Linear Regression
from sklearn.metrics import mean_squared_error # MSE
from sklearn.linear_model import Ridge # Ridge regressiojn
from sklearn.linear_model import Lasso # Lasso regression
from sklearn.cross_validation import cross_val_score # Cross validation 
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [ ]:
#Extracting Basic info about Train and Test data frame
print ('Shape of train: ' + str(train_df.shape))
print ('Shape of test: ' + str(test_df.shape))

In [ ]:
train_df.columns

In [ ]:
test_df.columns

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
#Delete ID Feature as it is completely useless.
del train_df['Id']
del test_df['Id']

In [ ]:
#Statistics of Numerical Data
train_df.describe()

**Data Exploration **

In [ ]:
#Distribution of Response variable - 'Sales Price'
sbs.distplot(train_df['SalePrice'])

In [ ]:
#As the distribution is slightly right skewed, we make it almost normal using log transformation
# But we will do it afterwards.

train_df['LogSalePrice'] = np.log(train_df['SalePrice'])
sbs.distplot(train_df.LogSalePrice)
#Much perfect normalized thing.

In [ ]:
#Look for corelation between dependent variables and independent variable
corr = train_df.corr()
corr = corr['SalePrice']
corr[np.argsort(corr, axis=0)[::-1]]

Following are the variables with corr > 0.5, this gives some intuition of their involvement in sales price.
* SalePrice        
* OverallQual      
* GrLivArea        
* GarageCars       
* GarageArea       
* TotalBsmtSF      
* FullBath         
* 1stFlrSF         
* YearBuilt        
* YearRemodAdd     
* GarageYrBlt      
* TotRmsAbvGrd    

Following two have slightly negative coefficient of correlation, shows iverse relation with sales price.

* KitchenAbvGr    
* EnclosedPorch  

In [ ]:
corrMatrix=train_df[["SalePrice","OverallQual","GrLivArea","GarageCars",
                  "GarageArea","GarageYrBlt","TotalBsmtSF","1stFlrSF","FullBath",
                  "TotRmsAbvGrd","YearBuilt","YearRemodAdd"]].corr()

plt.pyplot.figure(figsize=(10, 10))

sbs.heatmap(corrMatrix,vmax=.8, linewidths=0.01,
            square=True,annot=True,cmap='plasma',linecolor="white")

To reduce the multicollinearity, i.e. high correlation betweeen dependent variable we choose certain features from heatmap such that there is no multicollinearity while training our model.

We look at all the yellow boxes, i.e. with corr coef > 0.8. So, we find such pairs to be: 
* (TotRmsAbvGrd, GrLivArea)
* (YearBuilt, GarageYrBuilt)
* (1stFlrSF, TotalBsmntSF)
* (GarageAreas, GarageCars)

To reduce the effect of multicollinearity we will simply Remove one feature from each of the pair. (Afterwards, ofcourse.)

### Missing Value Imputation

In this we can see Only one variable(GarageyrrBlt) has null values which is to be used in training our data set, we impute these values. These are years which have to be integers for sure. Is it good to assume that Garage is  built on the same day as home? 

In [ ]:
train_df[train_df['GarageYrBlt'] == train_df['YearBuilt']].OverallQual.count()

1089 is quite large number of total, so we should assume that most of the garages are built with building only, so we reaplace empty values with YearBuilt.

In [ ]:
train_df.GarageYrBlt.fillna(train_df['YearBuilt'], inplace= True)
test_df.GarageYrBlt.fillna(train_df['YearBuilt'], inplace= True)

In [ ]:
null_columns=train_df.columns[train_df.isnull().any()]
train_df[null_columns].isnull().sum()

##### Lot Frontage

In [ ]:
#259 Missing value, there is another feature which is related to Lot, i.e. LotArea.
# First see correlation between them
train_df[['LotArea', 'LotFrontage']].corr()

In [ ]:
#Correlation is very less, let's try with sqrt of LotArea
train_df['sqrtLotArea'] = np.sqrt(train_df['LotArea'])
test_df['sqrtLotArea'] = np.sqrt(test_df['LotArea'])

In [ ]:
train_df[['sqrtLotArea', 'LotFrontage']].corr()

In [ ]:
#correlation is good, replace NAN by sqrtLotArea
train_df.LotFrontage.fillna(train_df['sqrtLotArea'], inplace = True)
test_df.LotFrontage.fillna(test_df['sqrtLotArea'], inplace = True)

### ALLEY

In [ ]:
#Most of the values are Null replace by 'None'
train_df.Alley.fillna('None', inplace= True)
test_df.Alley.fillna('None', inplace= True)

#### MAS VNR TYPE AND AREA

In [ ]:
cols = ['MasVnrType','MasVnrArea']
for items in cols:
    if train_df[items].dtype == 'object':
        train_df[items].fillna('None', inplace = True)
    elif train_df[items].dtype == 'float64':
        train_df[items].fillna(0.0, inplace = True)
for items in cols:
    if test_df[items].dtype == 'object':
        test_df[items].fillna('None', inplace = True)
    elif test_df[items].dtype == 'float64':
        test_df[items].fillna(0.0, inplace = True)

#### Basement

In [ ]:
#Almost anything with no basement have NAN value, replace it ny None

cols =[ u'BsmtQual',
       u'BsmtCond', u'BsmtExposure', u'BsmtFinType1',
       u'BsmtFinType2']

for col in cols:
    train_df[col].fillna('None', inplace = True)
for col in cols:
    test_df[col].fillna('None', inplace = True)

#### Garage

In [ ]:
#Since all the cols with NAN is correcponding to houses with ni garage, we replace NAN by None
cols = ['GarageType',
'GarageFinish',
'GarageQual',
'GarageCond' ]

for col in cols:
    train_df[col].fillna('None', inplace = True)
for col in cols:
    test_df[col].fillna('None', inplace = True)
    

#### Electrical

In [ ]:
# Only one value needs to fill up, so replace by mode
train_df.Electrical.fillna('SBrkr', inplace = True)
test_df.Electrical.fillna('SBrkr', inplace = True)

#### FirePlaceQu

In [ ]:
train_df.FireplaceQu.fillna('None', inplace = True)
test_df.FireplaceQu.fillna('None', inplace = True)

#### Pools, Fence & Misc

In [ ]:
#Pools definitely may have effect on prices. Although not much houses have pools, but for safety let us replace NAN by None
train_df.PoolQC.fillna('None', inplace= True)
train_df.Fence.fillna('None', inplace= True)
train_df.MiscFeature.fillna('None', inplace= True)
test_df.PoolQC.fillna('None', inplace= True)
test_df.Fence.fillna('None', inplace= True)
test_df.MiscFeature.fillna('None', inplace= True)

# Outlier Detection and Handling 

In [ ]:
# We use tukey method i.e. anything outside IQR * 1.5 is considered outlier, but it is removing som many points 
# So we are going to use IQR * 3
def Outlier(col, train_df):
    Q3 = train_df[col].describe().iloc[6]  
    Q1 = train_df[col].describe().iloc[4]
    IQR = Q3 - Q1
    Tukey_coeff = 3*IQR
    Lower_bound = Q1 - Tukey_coeff
    Upper_bound = Q3 + Tukey_coeff
    train_df = train_df[(train_df[col] > Lower_bound) & (train_df[col] < Upper_bound)]
    return train_df

In [ ]:
#We reamove Outliers from main numeric data which influence our model
col = ["SalePrice","OverallQual","GrLivArea","GarageCars",
                  "GarageArea","GarageYrBlt","TotalBsmtSF","1stFlrSF","FullBath",
                  "TotRmsAbvGrd"]
for cols in col:
    train_df = Outlier(cols, train_df)

In [ ]:
print (train_df.shape)
print (test_df.shape)

## Effect of different features on Sale Price


In [ ]:
sbs.pairplot(train_df[col[0:4]])

* POlynomial or quadratic relationship can be seen b/w Overall Quality and sales price.
* Linear relationship between GrLivArea and Sales Price
* Median of sales price also increases with in GragesCars
* Most garages have capacity of 2

In [ ]:
sbs.pairplot(train_df[['SalePrice', "GarageArea","GarageYrBlt","TotalBsmtSF"]])

* Salesprice increases with Garage area and BsmntSF, almost linearly.
* Prices increase quadrartically as newly constructed the house is.
* Building garages became more popular as passage of time.

In [ ]:
plt.pyplot.figure(figsize = (20,20))
train_df.boxplot(column='SalePrice', by = 'Neighborhood', figsize = (12,12), fontsize = 0.5)

## few nbd have really high prices of property, so must use nbd in our modeling.

In [ ]:
catg_var = train_df.dtypes[train_df.dtypes == "object"].index
for catg in list(catg_var) :
    bp = train_df.boxplot(column=['LogSalePrice'], by=[catg])

Upon seeing the relationship between categorical and sale prices, we can see that few cat. variables have very much impact 
on sale price so we will use only those features in our analysis.: 
    
'Neighborhood', 'Condition2', 'MasVnrType', 'ExterQual', 'BsmtQual','CentralAir', 'Electrical', 'KitchenQual', 'SaleType'

### Encoding Categorical variables to numerical

In [ ]:
cat = ['Neighborhood', 'Condition2', 'MasVnrType', 'ExterQual', 'BsmtQual','CentralAir', 'Electrical', 'KitchenQual', 'SaleType']
lb_make = LabelEncoder()
for col in cat:
    train_df["num" + col] = lb_make.fit_transform(train_df[col].astype(str))
for col in cat:
    test_df["num" + col] = lb_make.fit_transform(test_df[col].astype(str))

In [ ]:
print (train_df.shape)
print (test_df.shape)

In [ ]:
#correlation between categorical and target variable
cat = ['Neighborhood', 'Condition2', 'MasVnrType', 'ExterQual', 'BsmtQual','CentralAir', 'Electrical', 'KitchenQual', 'SaleType']
numcat = ['num'+item for item in cat]
numcat.append('LogSalePrice')
corr = train_df[numcat].corr()

In [ ]:
plt.pyplot.figure(figsize=(10, 10))
sbs.heatmap(corr,vmax=.8, linewidths=0.01,
            square=True,annot=True,cmap='plasma',linecolor="white")

* numExterQual, numBsmtQual, numKitchenQual are highly related to sales price, but all of the are correlated to each other also so we only use one of them to reduce multicollinearity.
* numCentralAir is also related and will be used for model training.

In [ ]:
#Total Columns which we are considering for model evaluation:
hero_col = ["OverallQual","GrLivArea",
                  "GarageArea","TotalBsmtSF","FullBath",
                  "YearBuilt","YearRemodAdd","numKitchenQual", 'numCentralAir','numNeighborhood', 'LogSalePrice']

In [ ]:
X = train_df[hero_col[:-1]]
y = train_df[hero_col[-1]]
X_maintest = test_df[hero_col[:-1]]
X_maintest.dropna(inplace=True)

In [ ]:
# Test train split of train_df
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, random_state=42, test_size=.33)

## OLR, ordinary linear regression.

In [ ]:
model = LinearRegression()
lr = model.fit(X_train, y_train)
b = lr.score(X_test, y_test)  
print ('R^2 error is %s.'% b)
mean_s_e = mean_squared_error(lr.predict(X_test), y_test)
print ('MSE is %s.' % mean_s_e)

In [ ]:
residuals = lr.predict(X_test) - y_test
fig = sbs.jointplot( lr.predict(X_test), residuals,kind='reg')
fig.set_axis_labels('Fitted','Residuals')

* Error is almost normally distributed.
* No pattern observed, whatsover. No transformation needed.
* Plot is not funnel shapped, Heteroskedasticity is not there.

### Using Cross validation to see possible overfitting

In [ ]:
#New Model training which will include all the observation for training.
New_lr = LinearRegression()
New_model = New_lr.fit(X,y)
scores = cross_val_score(New_lr, X, y , cv =10, scoring= 'r2')
#SE_scores = - cross_val_score(New_lr, X, y, cv =10, scoring= 'mean_squared_error')
print ('Mean of all the R^2 error after 10 folds of CV is %s.'% scores.mean())
#rint 'Mean of MSE after 10 folds of CV is %s.' % MSE_scores.mean()

##### Ridge Regression, finding optimal parameter of ridge regression using MSE


In [ ]:
R2 = []
alphas = []
mse = []
for i in range (-2,3):
    alpha = 10**i      # Range of alphas 
    rm = Ridge(alpha=alpha)   
    ridge_model = rm.fit(X_train, y_train)
    preds_ridge = ridge_model.predict(X_test)    #Training and predicting Ridge model
    plt.pyplot.scatter(preds_ridge, y_test, alpha=.75, color='b')
    plt.pyplot.xlabel('Predicted Price')
    plt.pyplot.ylabel('Actual Price')
    plt.pyplot.title('Ridge Regularization with alpha = {}'.format(alpha))    #Plotting actual vs predicted price for difft alphas
    R2.append(rm.score(X_test, y_test))
    mse.append(mean_squared_error(y_test, preds_ridge))
    #rmse = np.sqrt(mse)
    alphas.append(alpha)
    print ('R2 Error for alpha = %s is %s'%(alpha, rm.score(X_test, y_test)))
    plt.pyplot.show()

In [ ]:
plt.pyplot.plot(alphas, mse)

##### Minimized MSE is at 10.. So, value of alpha can be choosen to be 10.

Lasso Regression 

In [ ]:
R2 = []
alphas = []
mse = []
for i in range (-5,3):
    alpha = 10**i
    ls = Lasso(alpha=alpha)
    lasso_model = ls.fit(X_train, y_train)
    preds_lasso = lasso_model.predict(X_test)
    plt.pyplot.scatter(preds_lasso, y_test, alpha=.75, color='b')
    plt.pyplot.xlabel('Predicted Price')
    plt.pyplot.ylabel('Actual Price')
    plt.pyplot.title('Lasso Regularization with alpha = {}'.format(alpha))
    R2.append(ls.score(X_test, y_test))
    mse.append(mean_squared_error(y_test, preds_lasso))
    #rmse = np.sqrt(mse)
    alphas.append(alpha)
    print ('R2 Error for alpha = %s is %s'%(alpha, ls.score(X_test, y_test)))
    plt.pyplot.show()
R2

In [ ]:
plt.pyplot.plot(mse, alphas)

### MInimum of MSE can be found at alpha = 0.001

#### Ridge Regression CV with alpha = 10

In [ ]:
New_Ridge_lr = Ridge(alpha = 10)
New_model = New_Ridge_lr.fit(X,y)
scores = cross_val_score(New_Ridge_lr, X, y , cv =10, scoring= 'r2')
#SE_scores = - cross_val_score(New_lr, X, y, cv =10, scoring= 'mean_squared_error')
print ('Mean of all the R^2 error after 10 folds of CV is %s.'% scores.mean())
#rint 'Mean of MSE after 10 folds of CV is %s.' % MSE_scores.mean()

#### Lasso Regression CV with alpha = 0.001

In [ ]:
New_Lasso_lr = Lasso(alpha = 0.001)
New_model = New_Lasso_lr.fit(X,y)
scores = cross_val_score(New_Lasso_lr, X, y , cv =10, scoring= 'r2')
#SE_scores = - cross_val_score(New_lr, X, y, cv =10, scoring= 'mean_squared_error')
print ('Mean of all the R^2 error after 10 folds of CV is %s.'% scores.mean())
#rint 'Mean of MSE after 10 folds of CV is %s.' % MSE_scores.mean()

## We've transformed sale prices using log transformation. Make sure to invert the prediction using np.log() transformation to get original scores.